In [ ]:
#instalando a biblioteca Llama index
!pip install -q llama-index

In [ ]:
# biblioteca para permitir a leitura de arquivos
from llama_index.core import SimpleDirectoryReader
documentos = SimpleDirectoryReader(input_dir='documentos')

In [ ]:
documentos.input_files


In [ ]:
docs = documentos.load_data()

In [ ]:
docs

In [ ]:
len(docs)

In [ ]:
# importando a biblioteca para fazer a parse das informações
from llama_index.core.node_parser import SentenceSplitter

node_parser= SentenceSplitter(chunk_size=1200)

nodes = node_parser.get_nodes_from_documents(docs,show_progress=True)

In [ ]:
nodes



In [ ]:
len(nodes)

In [ ]:

#instalaçao do hugging face
!pip install -q llama-index-embeddings-huggingface

In [ ]:
#importa a biblioteca para fazer o embeding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [ ]:
# Classe personalizada para integrar ao Chroma DB

class ChromaEmbeddingWrapper:
  def __init__(self,model_name:str): # inicializa com o modelo embedding
    self.model = HuggingFaceEmbedding(model_name = model_name)


  def __call__(self,input): # Converte a entrada para o formato compativel com o HuggingFace
    return self.model.embed(input)


In [ ]:
embed_model_chroma= ChromaEmbeddingWrapper(model_name='intfloat/multilingual-e5-large')

In [ ]:
#instalaçao do chroma db
!pip install -q llama-index-vector-stores-chroma

In [ ]:
#importando chroma db

import chromadb
db = chromadb.PersistentClient(path='./chroma_db')
chroma_client = db
collection_name = 'documentos_doc_info_agricultura'

try:
 chroma_collection = chroma_client.get_or_create_collection(
        name=collection_name,
        embedding_function=embed_model_chroma)

except Exception as e :
  print(f'Erro ao criar coleçao {e}')

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext


In [ ]:
#criando a coleçao do banco de dados
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
embed_model = HuggingFaceEmbedding(model_name='intfloat/multilingual-e5-large')

In [ ]:
from llama_index.core import VectorStoreIndex

In [ ]:
index = VectorStoreIndex(nodes, storage_context=storage_context,embed_model = embed_model)

In [ ]:
from llama_index.core import load_index_from_storage
index = load_index_from_storage(storage_context,embed_model = embed_model)

In [ ]:
from google.colab import userdata
Groq_api =userdata.get('api_aula07')

In [ ]:
!pip install -q llama-index-llms-groq

In [ ]:
from llama_index.llms.groq import Groq

In [ ]:
llms = Groq(model = 'llama3-70b-8192',api_key=Groq_api )

In [ ]:
chat_engine = index.as_chat_engine(mode='context',llm = llms)

In [ ]:

chat_engine.chat_history

In [ ]:
from llama_index.core.memory import ChatSummaryMemoryBuffer

In [ ]:
memory = ChatSummaryMemoryBuffer(llm=llms,token_limit=256)

In [ ]:
chat_engine= index.as_chat_engine(
    chat_mode = 'context',
    llm=llms,
    memory=memory,
    system_prompt=('''Voce é especialista em agriculruta da Agrotrack, uma emoresa no ramo de agricultura,
sua funçao é tirar duvidas de forma simpatica e natural sobre tipos de plantio, sementes e o que mais envolver a agricultura.'''))

In [ ]:
response = chat_engine.chat('Ola')
print(response)

In [ ]:
response = chat_engine.chat('Voce pode me dar mais detalhes sobre o solo?')
print(response)

In [ ]:
response = chat_engine.chat('Quais sãos os seus tipos?')
print(response)

In [ ]:
memory.get()

In [ ]:
#resetando o chat
chat_engine.reset()

In [ ]:
response = chat_engine.chat('Quais sãos os seus tipos?')
print(response)

In [ ]:
# implementando a interface com o Gradio
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
# Função para conversar com o chatbot
def converse_com_bot(message, chat_history):
    # Simulando uma resposta do chatbot (substitua com sua lógica real de chatbot)
    response = chat_engine.chat(message)  # Altere conforme necessário

    if chat_history is None:
        chat_history = []

    chat_history.append({"role": "user", "content": message})
    chat_history.append({"role": "assistant", "content": response.response})  # Ajuste conforme necessário

    return "", chat_history

# Função para resetar o chatbot
def resetar_chat():
    chat_history = []  # Reseta o histórico
    return chat_history

In [ ]:
# Criando a interface Gradio
with gr.Blocks() as app:
    gr.Markdown('# Chatbot da AgroTrack')
    chatbot = gr.Chatbot(type='messages')
    msg = gr.Textbox(label='Digite a sua mensagem')
    limpar = gr.Button('Limpar')

    msg.submit(converse_com_bot, [msg, chatbot], [msg, chatbot])
    limpar.click(resetar_chat, None, chatbot, queue=False)

    app.launch(debug=True)